**F21BC**

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [14]:
df = pd.read_excel('concrete+compressive+strength/Concrete_Data.xls')


**Viewing the first few instances of the data**

In [15]:
df.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


*Viewing the dimensionality of the dataset* 

In [21]:
df.shape

(1005, 9)

In [16]:
df.dropna(inplace = True)
df.isnull().sum()

Cement (component 1)(kg in a m^3 mixture)                0
Blast Furnace Slag (component 2)(kg in a m^3 mixture)    0
Fly Ash (component 3)(kg in a m^3 mixture)               0
Water  (component 4)(kg in a m^3 mixture)                0
Superplasticizer (component 5)(kg in a m^3 mixture)      0
Coarse Aggregate  (component 6)(kg in a m^3 mixture)     0
Fine Aggregate (component 7)(kg in a m^3 mixture)        0
Age (day)                                                0
Concrete compressive strength(MPa, megapascals)          0
dtype: int64

In [19]:
print(f'Duplicate rows: {df.duplicated().sum()}')

Duplicate rows: 25


In [18]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**Scaling and Encoding**

In [23]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1].values  
y = df.iloc[:, -1].values   

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42) 


In [54]:
scaler = StandardScaler()

#intialise training and test sets
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)        

In [59]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.634274   -0.81866937  0.67425626 -1.10040309  0.46818459  0.40876467
   1.63220543  0.87023165]
 [ 1.37649573  0.41224912 -0.88193591 -1.37711434  1.82041152 -1.58177583
   1.43038796  0.72634905]
 [-0.30958693 -0.81866937  1.05737381 -1.86207219  1.0375433   1.41873705
   0.37528277 -0.28082919]
 [-1.01413233 -0.66107011  1.7905842  -1.222118   -0.32509872  0.38958798
   1.05465742 -0.28082919]
 [-1.07365672 -0.81866937  1.64949275 -0.28548372 -0.26434398  1.05821526
   0.12071615  0.87023165]]


**Artificial Neural Network (ANN)**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [26]:
class activationFunction:
    def logisticFunction(x):
        return 1 / (1 + np.exp(-x))

    def reLuFunction(x):
        return np.maximum(0, x)

    def hyperbolicFunction(x):
        return np.tanh(x)

    def leakyReLU(x, alpha=0.01):
        return np.maximum(x, alpha * x)

    def elu(x, alpha=1.0):
        return np.where(x > 0, x, alpha * (np.exp(x) - 1))

class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        if len(activationFunction) != len(layerSize) - 1:
            raise ValueError("Number of activation functions must be one less than number of layers.")

        self.layerSize = layerSize
        self.activationFunction = activationFunction
        self.weights = [np.random.randn(layerSize[i], layerSize[i + 1]) for i in range(len(layerSize) - 1)]
        self.biases = [np.random.randn(1, layerSize[i + 1]) for i in range(len(layerSize) - 1)]

    def forwardPropagation(self, x):
        output = x
        for i in range(len(self.weights)):
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output = self.activationFunction[i](matrix_total)  # Apply the ith activation function
        return output

*Layer size is structure to be 8 input layers, 16 neurons in first hidden layer, 8 for the second hidden layer, and 1 for the output layer.*
*This is to check the predicted values generated after forward propagation from the ANN*

In [48]:
layer_sizes = [8, 16, 8, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reLuFunction,
    activationFunction.hyperbolicFunction,
]

ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)

x_example = np.random.rand(8, 8)  # 8samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)


ANN Output (Sample Predictions): [[0.75832783]
 [0.95719977]
 [0.99574793]
 [0.9929544 ]
 [0.99024769]
 [0.91919397]
 [0.99975592]
 [0.94464942]]


**Loss Function**
*Since the problem domian is regression, MSE is utilised as the loss function*

In [60]:
class lossFunction:
    def evaluate(self, y_pred, y_train):
        pass
class MeanSquaredError(lossFunction):
    def evaluate(self, y_pred, y_train):
        return np.mean((y_pred - y_train) ** 2)

In [62]:
loss_function = MeanSquaredError()

#output of forward propagation
y_pred = ann.forwardPropagation(X_train_scaled)

#use the output to calculate the loss function 
print("Loss:", loss_function.evaluate(y_pred, y_train))

Loss: 1447.58422660566


**Implement PSO Algorithm** 

In [ ]:
class Particle:
    def __init__(self, vectorSize):
        
        #self.particlePosition=numpy.random.rand(vectorSize)
        #self.particleVelocity=numpy.random.rand(vectorSize)    i think this is wrong. i think it should be initialising it from zero 
        
        self.particlePosition = np.zeros(vectorSize)
        self.particleVelocity = np.zeros(vectorSize)
        self.bestPosition=np.zeros(vectorSize)
        self.bestFitness = float('inf')  #best fitness 

    def updateBestPosition(self, fitness):
        
        if fitness < self.best_fitness:  # Minimize fitness
            self.best_fitness = fitness
            self.best_position = np.copy(self.position)

        #we probably need a function for update velocity 
    

In [ ]:
from Particle import Particle
class ParticleSwarmOptimisation:
    # initialise variables, use init function: constructor method
    def __init__(self,swarmSize,alpha,beta,delta,omega,jumpSize,informantCount,vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.omega = omega
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')


        # assign informants
        def initInformants(informantCount,particleArray):
            pass

        # Example of assessFitness
        def assessFitness(particle, X, y, loss_function,predictions):
            # Implement forward propagation for the ANN represented by this particle
            # Use particle's position as ANN weights/biases
            # Calculate the loss (or error) using the provided loss_function
            # Return the computed fitness
            predictions = particle.forward_prop(X)  # Suppose each particle has a forward_prop method
            fitness = loss_function(predictions, y)
            return fitness

        
        def psoOptimisation(swarmSize,alpha,beta,delta,omega,jumpSize,informantCount,vectorSize):
            # stores all of the particles
            particleArray=[]
            for i in range(swarmSize):
                particleArray.append(Particle(vectorSize))
            best=None
            #initialising informants for the particles
            '''write code'''
            while(True): #will change to do while loop
                # compare fitness
                for p in particleArray:
                    particleFitness=assessFitness(p)
                    bestFitness=assessFitness(best)
                    if best is None or particleFitness<bestFitness:
                        best=p
                for p in particleArray:
                    previousBest=p.bestPosition
                    #informantsBest=
                    allBest=best.bestPosition








